In [11]:
import pandas


"Fatal failure while accessing filings: Unable to get data for URL 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=msft&type=10-k&owner=include&count=40'. Surpassed maximum try count of 10"